In [ ]:
import os
import matplotlib
import matplotlib.pyplot as plt
import PIL
from PIL import Image                # working with images
import shutil


# coastvision
# from coastvision import geospatialTools
# from coastvision.classifier import data_annotation_tool 
from coastvision import coastvision
from coastvision import supportingFunctions
from coastvision import dataSelectionTools

# Digitize shoreline reference
Here you enter the region, site, and the timeperiod you want to create a reference shoreline for (timeperiod can be None). Then you will choose an image from this timeperiod to hand digitize.

In [ ]:
import importlib
importlib.reload(dataSelectionTools) 
importlib.reload(supportingFunctions) 

# this can be easily set up to run in a loop
region = 'kauai'
site = 'kauai0001'
# dataSelectionTools.get_reference_sl(region, site, timeperiod=[[2010, 2025], [1, 12]])
dataSelectionTools.get_reference_sl(region, site, timeperiod=None)

# dataSelectionTools.get_reference_sl(region, site, (2010,2029))

#dataSelectionTools.get_reference_sl('oahu', site, (2010,2019))

In [ ]:
from coastvision import dataSelectionTools
import importlib
importlib.reload(dataSelectionTools)
transects = dataSelectionTools.get_transects('trainingdata', 'cadizspain')

In [ ]:
timeperiods = [None, (2010, 2019), (2020, 2029)]
timeperiods = [(2020, 2029)]

for timeperiod in timeperiods:

    for region in os.listdir(os.path.join(os.getcwd(), 'data')):
        for sitename in os.listdir(os.path.join(os.getcwd(), 'data', region)):
            if sitename != 'oahu0194':
                continue
            
            pth = os.path.join(os.getcwd(), 'data', region, sitename,)
            #validTifs = dataSelectionTools.select_tif_images(pth, timeperiod)

pth

# select reference shorelines
Here you look through tif images from the specified time period and then run the model on those images. Then you look through the output and decide if you want to use one of the outputed contours as the reference shoreline

In [ ]:
timeperiods = [None, (2010, 2019), (2020, 2029)]
timeperiods = [(2020, 2029)]

for timeperiod in timeperiods:

    for region in os.listdir(os.path.join(os.getcwd(), 'data')):
        for sitename in os.listdir(os.path.join(os.getcwd(), 'data', region)):
            if sitename != 'oahu0194':
                continue
            
            pth = os.path.join(os.getcwd(), 'data', region, sitename,)
            validTifs = dataSelectionTools.select_tif_images(pth, timeperiod)

            # %matplotlib inline
            timestampList = list()
            for validTif in validTifs:
                itemID = '_'.join(validTif.split('_')[:-4])
                # should check if classification already exists
                %matplotlib inline
                coastvision.run_coastvision_single(region, sitename, itemID, justShorelineExtract=True, dataProducts=True)
                timestampList.append('_'.join(itemID.split('_')[:-1]))
                
                # NOTE: THERE ARE PLOTS IN run_coastvision_single THAT WILL BE


            imgFolder = os.path.join(os.getcwd(), 'outputs', region, sitename, 'data_products', 'contours')

            # select reference shoreline
            contourpngFileList = dataSelectionTools.select_images(imgFolder, timeperiod=timeperiod, filterList=timestampList, onlySelectOne=False)

            # The user can select multiple and the last one selected will become the reference shoreline
            for contourPlotFn in contourpngFileList:
                # right now this should just save the last shoreline the user said was good
                fn_start = contourPlotFn.split('_rgb')[0].replace('-', '_')
                year = fn_start.replace(sitename, '')
                print(fn_start)
                year = int(year[1:5])
                print(year)
                contourFileName = os.path.join(os.getcwd(), 'outputs', region, sitename, 'shoreline', 'contours', f'{fn_start}_contour.geojson')
                if timeperiod is None:
                    # also use as the default shoreline
                    shutil.copyfile(contourFileName, os.path.join(os.getcwd(), 'user_inputs', region, sitename, f'{sitename}_shoreline.geojson'))
                    shutil.copyfile(contourFileName, os.path.join(os.getcwd(), 'user_inputs', region, sitename, f'{sitename}_shoreline_{year}_{timeperiod[0]}-{timeperiod[1]}.geojson'))
                else:
                    shutil.copyfile(contourFileName, os.path.join(os.getcwd(), 'user_inputs', region, sitename, f'{sitename}_shoreline_{year}_{timeperiod[0]}-{timeperiod[1]}.geojson'))

# select training data for satCNN

In [ ]:
for region in os.listdir(os.path.join(os.getcwd(), 'data')):
    if not 'oahu' == region:
        continue
    for sitename in os.listdir(os.path.join(os.getcwd(), 'data', region)):

        print(sitename)
        imgFolder = os.path.join(os.getcwd(), 'outputs', region, sitename, 'data_products', 'contours')
        
        # select training data for is shoreline valid CNN
        retuDict = dataSelectionTools.select_images(imgFolder, timeperiod=None, filterList=None, onlySelectOne=False, returnNonSelected=True) # we only want to look through
        
        # copy images to satCNN trianing data
        if type(retuDict) == list:
            continue # the user either didnt select any or only selected of one class NEED TO FIX SO THAT WE CAN RECOGNIZE THAT
        for key, item in retuDict.items():
            if key == 'selected':
                classDir = 'validshoreline'
            elif key == 'notselected':
                classDir = 'invalidshoreline'
            newTrainingDataDir = os.path.join(os.getcwd(), 'satCNN', 'data', 'shorelineQAQC', 'train', classDir)
            for fn in item:
                shutil.copy(os.path.join(imgFolder, fn), os.path.join(newTrainingDataDir))